#### import packages

In [1]:
import collections
import os
import shutil
import sys
from errno import errorcode
import socket
from urllib3.connection import HTTPConnection
import requests
from getpass import getpass
import mysql.connector
from requests import exceptions
import pysftp
import collections

#### Function to setup database connection

In [2]:
def getDbServer():
    return 'rslims.jax.org'


def getDbUsername():
    return 'dba'


def getDbPassword():
    return 'rsdba'


def getDbSchema():
    return 'rslims'    

#### Connect to database

In [4]:
user = getDbUsername()
pwd = getDbPassword()
server = getDbServer()
schema = getDbSchema()

try:
    conn = mysql.connector.connect(host=server, user=user, password=pwd, database=schema)

except mysql.connector.Error as err1:
    if err1.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Wrong user name or password passed")

    elif err1.errno == errorcode.ER_BAD_DB_ERROR:
        print("No such schema")

    else:
        error = str(err1.__dict__["orig"])
        print(error)

except ConnectionError as err2:
    print(err2)

In [5]:
conn

#### Set a higher timeout for connecting to Omero

In [6]:
HTTPConnection.default_socket_options = (HTTPConnection.default_socket_options + [
        (socket.SOL_SOCKET, socket.SO_SNDBUF, 1000000),
        (socket.SOL_SOCKET, socket.SO_RCVBUF, 1000000)
    ])

#### Connect to Omero

In [7]:
session = requests.Session()
url = "https://omeroweb.jax.org/api/"
response = session.get(url, verify=True)

content = response.json()['data']
forms = content[-1]
base_url = forms['url:base']

#### Get metadata of the webpage

In [8]:
r = session.get(base_url)
urls = r.json()
servers_url = urls['url:servers']

In [9]:
servers_url

'https://omeroweb.jax.org/api/v0/servers/'

#### Get the CSRF token for logging in

In [10]:
token_url = urls["url:token"]
csrf = session.get(token_url).json()["data"]

In [11]:
csrf

'Xkf467v2GY0zX2l4OOxExTv5t1H6XT5CnxQVoRWmSZqyDeu46dTM3BwVsWQkJbbg'

#### List the servers available to connect to

In [12]:
servers = session.get(servers_url).json()['data']
servers = [s for s in servers if s['server'] == 'omero']
if len(servers) < 1:
    raise Exception("Found no server called 'omero'")
server = servers[0]

In [13]:
server

{'id': 1, 'host': 'ctomero01lp', 'port': 4064, 'server': 'omero'}

#### Sign in with your credentials

In [14]:
login_url = urls['url:login']
session.headers.update({'X-CSRFToken': csrf,
                        'Referer': login_url})
payload = {'username': input("Username: "),
            'password': getpass("Password: "),
            'server': server['id']
            }
r = session.post(login_url, data=payload)

Username: chent
Password: ········


In [15]:
r.json()

{'success': True,
 'eventContext': {'sessionId': 1214496,
  'sessionUuid': '9e006191-357e-4eaa-9f73-14cdf2169403',
  'userId': 1188,
  'userName': 'chent',
  'groupId': 403,
  'groupName': 'KOMP_histopath',
  'isAdmin': False,
  'eventId': -1,
  'eventType': 'User',
  'memberOfGroups': [403, 1, 3, 153],
  'leaderOfGroups': []}}

#### Check exceptions

In [18]:
login_rsp = r.json()
try:
    r.raise_for_status()
except exceptions.HTTPError as e:
    # Whoops it wasn't a 200
    print("Error {}".format(e))
    raise
assert login_rsp['success']
print(f'login response: {login_rsp}')

login response: {'success': True, 'eventContext': {'sessionId': 1214496, 'sessionUuid': '9e006191-357e-4eaa-9f73-14cdf2169403', 'userId': 1188, 'userName': 'chent', 'groupId': 403, 'groupName': 'KOMP_histopath', 'isAdmin': False, 'eventId': -1, 'eventType': 'User', 'memberOfGroups': [403, 1, 3, 153], 'leaderOfGroups': []}}


#### Map to store info of an image, with procedure instance key as key, list of tuple (url, testcode) as value. 

In [19]:
urlMap = collections.defaultdict(list)

#### Read the .sql file

In [21]:
sqlfile = 'omeros.sql'
fptr = open(sqlfile, "r")
sql = fptr.read()

In [22]:
sql

"SELECT\n    ProcedureDefinition, ProcedureDefinition.ExternalID, OutputName, OutputValue, _procedureInstance_Key as TestCode\nFROM\n    proceduredefinition\n        INNER JOIN\n    proceduredefinitionversion USING (_ProcedureDefinition_key)\n        INNER JOIN\n    ProcedureInstance USING (_ProcedureDefinitionVersion_key)\n        INNER JOIN\n        outputInstanceSet USING (_ProcedureInstance_key)\n        inner join\n    outputinstance USING (_outputInstanceSet_key)\n        INNER JOIN\n    Output USING (_Output_key)\nWHERE\nCHAR_LENGTH(OutputValue) > 0 AND\nOutputValue LIKE '%omeroweb%' AND\n    outputinstance._Output_key IN (1842 , 2148, 4628);"

#### Query database

In [23]:
cursor = conn.cursor(buffered=True, dictionary=True)
cursor.execute(sql)
queryResult = cursor.fetchall()

In [24]:
queryResult

[{'ProcedureDefinition': 'Gross Pathology',
  'ExternalID': 'IMPC_PAT_002',
  'OutputName': 'Images',
  'OutputValue': 'https://omeroweb.jax.org/webgateway/img_detail/69844',
  'TestCode': 3097662},
 {'ProcedureDefinition': 'Gross Pathology',
  'ExternalID': 'IMPC_PAT_002',
  'OutputName': 'Images',
  'OutputValue': 'https://omeroweb.jax.org/webgateway/img_detail/69845',
  'TestCode': 3097662},
 {'ProcedureDefinition': 'Gross Pathology',
  'ExternalID': 'IMPC_PAT_002',
  'OutputName': 'Images',
  'OutputValue': 'https://omeroweb.jax.org/webgateway/img_detail/69846',
  'TestCode': 3097662},
 {'ProcedureDefinition': 'Gross Pathology',
  'ExternalID': 'IMPC_PAT_002',
  'OutputName': 'Images',
  'OutputValue': 'https://omeroweb.jax.org/webgateway/img_detail/69841',
  'TestCode': 3097662},
 {'ProcedureDefinition': 'Gross Pathology',
  'ExternalID': 'IMPC_PAT_002',
  'OutputName': 'Images',
  'OutputValue': 'https://omeroweb.jax.org/webgateway/img_detail/69842',
  'TestCode': 3097662},
 {'Pr

#### Parse the query result, and store them into ```urlMap```

In [25]:
 for dict_ in queryResult:
    # print(dict_)
    for key, val in dict_.items():

        if key == "ExternalID":
            link = dict_["OutputValue"]
            testCode = dict_["TestCode"]
            urlMap[val].append((link, testCode))

#### ```urlMap``` is

In [26]:
urlMap

defaultdict(list,
            {'IMPC_PAT_002': [('https://omeroweb.jax.org/webgateway/img_detail/69844',
               3097662),
              ('https://omeroweb.jax.org/webgateway/img_detail/69845',
               3097662),
              ('https://omeroweb.jax.org/webgateway/img_detail/69846',
               3097662),
              ('https://omeroweb.jax.org/webgateway/img_detail/69841',
               3097662),
              ('https://omeroweb.jax.org/webgateway/img_detail/69842',
               3097662),
              ('https://omeroweb.jax.org/webgateway/img_detail/69843',
               3097662),
              ('https://omeroweb.jax.org/webgateway/img_detail/69916',
               3097664),
              ('https://omeroweb.jax.org/webgateway/img_detail/69917',
               3097664),
              ('https://omeroweb.jax.org/webgateway/img_detail/69918',
               3097664),
              ('https://omeroweb.jax.org/webgateway/img_detail/69913',
               3097664),
      

#### Set up the root directory for downloading files

In [27]:
path = "/Users/chent/Desktop/KOMP_Project/download_from_omero/Omero_Files"

#### Unpack ```urlMap``` and start downloading

In [ ]:
for key, val in urlMap.items():
    
    dest = path + "/" + key
    try:
        os.mkdir(dest)

    except FileExistsError as e:
        print(e)

    for pair in val:
        link, test_code = pair[0], pair[1]
        print(link)
        omeroId = link.split("/")[-1].strip()
        images_url = urls['url:images'] + str(omeroId)
        print(images_url)
        # get the filename from OMERO GET https://omeroweb.jax.org/api/v0/m/images/nnn - then get the "Name"
        # attribute of the response.
        resp = session.get(images_url)
        j = resp.json()
        name = j["data"]["Name"].strip()
        frm, to = name.find("["), name.find("]")
        name = name.replace(name[frm:to+1], "")
        fName = str(test_code) + name
        # File name has junk in it. Like " []". Needs a tif extension instead.
        downloadFileUrl = base_url.replace("api/v0/", "webgateway/archived_files/download/")
        downloadFileUrl = downloadFileUrl + str(omeroId)

        """Starting to download files"""
        with session.get(downloadFileUrl, stream=True) as r:
            r.raise_for_status()
            with open(dest + "/" + fName, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    # If you have chunk encoded response uncomment if
                    # and set chunk_size parameter to None.
                    # if chunk:
                    f.write(chunk)
            f.close()
        
    """Send images to SFTP server"""
    fileList = os.listdir(path)
    cnopts = pysftp.CnOpts()
    cnopts.hostkeys = None

    for file in fileList:
        with pysftp.Connection(host="bhjlk01lp.jax.org", username="jlkinternal",
                                password="t1m3st4mp!", cnopts=cnopts) as sftp:
            sftp.cwd("/srv/ftp/images/"+key)
            sftp.put(file)
            sftp.close()
            
    """Empty directory after send all files related to one procedure key"""
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))